# Key Concepts and Definitions

This heading clearly introduces the definitions and explanations provided in the following section.
Before we dive into the notebook, let's take a moment to break down a few key concepts in a simple and friendly way. This will help make the rest of the material much easier to follow!

**What does "Latent" mean?**  
In data science, "latent" means something that is hidden or not directly visible. For example, in text analysis, there might be hidden themes or patterns in a collection of documents that are not obvious just by reading them. These hidden patterns are called "latent" because they are not directly observed but can be discovered using special methods.

**What is a "Topic"?**  
A "topic" is a group of words that often appear together and represent a common theme or subject in a set of documents. For example, words like "economy," "growth," and "production" might form a topic about economics. In topic modeling, the computer tries to find these groups of words (topics) that best describe the themes in the documents.

**What is "Sampling"?**  
"Sampling" is a way to make guesses or estimates by looking at a small part of the data instead of the whole thing. In topic modeling, sampling is used to repeatedly guess which topic a word belongs to, based on the current state of the model. Over many rounds, these guesses help the model discover the hidden topics in the documents. Think of it like drawing names from a hat many times to get a good idea of what names are in the hat.

**What is "Gibbs Sampling"?**  
Gibbs sampling is a special kind of sampling used to estimate hidden patterns when there are many unknowns. In topic modeling, Gibbs sampling works by going through each word in each document and guessing which topic it belongs to, based on the current guesses for all the other words. After each guess, the model updates its understanding of the topics. By repeating this process many times, the model gradually gets better at figuring out which words belong to which topics. It's like solving a big puzzle by making lots of small, smart guesses, one piece at a time.

## Topic modeling (LDA)
- It implements latent dirichlet allocation (a popular topic modeling approach)
- The model uses collapsed gibbs sampling (a faster inference model for topic modeling)

It operates in two steps.

*A) Preparing data (integer encoding documents)*  

*B) Performing topic modeling on integer encoded documents*

In [1]:
# Its vanilla implementation of Topic modeling that only uses basic tools:
# json - to read from and write to files in json format 
# numpy - for faster matrix operations 
# pandas - to read csv data
# string - to only keep English letters, removing puntuations and other characters
# random - to generate random numbers for initializing Markov-chain monte carlo, and 
#           and during algorithm working to avoid local optima


import json
import numpy as np
import random
import pandas as pd
import string

# A) Preparing data (integer encoding documents)

1. Read textual data
2. Generate integer encoding
3. Storing intemediate data

**Working with integers (representing words or unique tokens is much faster than the word strings itself)**

*At the end, the integers would be reversed back to their respective words*

## 1. Reading textual data
- Read raw text from .txt file having document per line
- Separate into list of documents
- Tokenize

1.1 Clean text by removing punctuations and characters othen than English letters \
1.2 Convert to lower case \
1.3 Tokenize 

In [2]:
en_stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

def clean_text(text):
    clean_text = text.lower()
    # cleaning documents by removing unwanted characters
    clean_text = "".join([char for char in text if char in string.ascii_lowercase])
    # cleaning documents by stopwords
    clean_text = [word for word in text.split(" ") if word not in en_stopwords and len(word) > 2]
    return clean_text

1.4 Read data from the file 

In [3]:
# Read input data: titles of BBC articles available on the following link
# https://github.com/vahadruya/Capstone-Project-Unsupervised-ML-Topic-Modelling/blob/main/Input_Data/input.csv

with open('config.json', 'r') as file:
    config = json.load(file)

with open(config["input-path"], 'r') as file:
    input_data = file.read().split('\n')




# Tokenize sentences into words
tokenized_documents = []
for document in input_data[:100]: # Considering only first 100 titles for the sake of demonstration
    document = clean_text(document)
    if len(document) > 2:
        tokenized_documents.append(document)
len(tokenized_documents)

100

## 2. Configuration

In the repository you can find `config.json`. The method provides the following configuration options to alter the behavior of the method from `config.json`

**numTopics:** Number of topics to extract from the dataset. The default value is 3. However, it generally depends on the nature of the data. Keeping the number of topics too few can only have the topics focused on broader concepts and cannot identify the specific topics. While keeping the number of topics too high results in noisy (incoherent) topics. Therefore, a suitable number of topics depends on the data and the type of analysis required. The default value here is 3.

**numAlpha**: This hyperparameter helps in deciding the probabilities of topics in a document. A higher value (above 1) will have too many topics with similar probabilities i.e., when the intended purpose is to get more topics per document. However, keeping the value too low will have only few promiment topics with very high probabilities as compared to others. A lower value (below 1) is used when only fewer topics are needed per document. A lower numAlpha value pushes higher probabilities higher and lower probabilities further lower. While a higher numAlpha value introduces a high bias due to which all probabilities converges to similar values. In this method we are using numAlpha value 1.

**numBeta**: This hyperparameter helps in deciding the probabilities of words in a topic. a higher value (above 1) will have too many words with similar probabilities i.e., when the intended purpose is to have more words representing a topic. However, keeping the value too low will have only few most prominent words with very high probabilities as compared to others. Generally, considering the size of vocabulary in a dataset, this value is kept smaller to determine only the most relevant words. In this method we are using numBeta value 0.01.

**numGSIterations**: Its the number of iterations of the inference technique (collapsed Gibbs sampling). Due to random initialization, there are more words switching their topics in the earlier iterations that keeps dropping in the coming iterations i.e., approaching the equilibrium state. Keeping the number of numGSIterations higher ensures that the words have settled down in their respective topics. Alternately, the difference between two consecutive iterations is also used to avoid unnecessary iterations when the words have already settled. In this method we only use the numGSIterations with value 1000.

**wordsPerTopic**: The number of top words to represent a topic. In this method we are using 10 most prominent words for each topic. Due to polysemy, it is possible that a word exist in different topics with different neighboring words highlighting its context. 

**text-doc-path**: path of input (raw text) file
**integer-encoded-doc-path**: path of integer encoded file. It is the intermediate file that topic modeling use. 
**integer-word-dict**

## 3. Generate Integer encoding
It preserves both frequency and position related information. The process involves assigning each unique token a dedicated integer id, preserving it in a dictionary for later retrieval, while rewriting documents by replacing with with their integer ids.

It makes the operations a lot faster as numbers are much faster to read/store and compare as compared to strings. 

The integer ids will be replaced with their original words at the end using stored dictionary files

3.1 Generate integer encoded documents \
3.2 Generate word-integer index and integer index-word dictionaries

In [4]:
# Create a dictionary of unique tokens and assign integers
dictionary = {}
revdictionary = {}
index = 0

#tokenized_documents = [[word for word in doc if word not in esw] for doc in tokenized_documents]

for doc in tokenized_documents:
    for word in doc:
        if word not in dictionary.keys():
            dictionary[word] = index
            revdictionary[index] = word
            index += 1

# Replace words in sentences with their corresponding integers
encoded_documents = [[dictionary[word] for word in doc] for doc in tokenized_documents]

### 4. Storing intermediate data
The integer encoded documents are stored in files
the word-to-id and id-to-word dictionaries are also stored

*It will help to avoid these steps, each time topic modeling is performed under different settings*

In [5]:
toStr = ''
for endoc in encoded_documents:
    toStr = toStr + '\t'.join(str(item) for item in endoc)
    toStr = toStr + '\n'
toStr = toStr[:-2]
file = open('data/integer-encoded-data.txt', 'w')
file.write(toStr)
file.close()

#write dictionary to file
file = open('data/dictionary.json', 'w')
file.write(json.dumps(dictionary))
file.close()
file = open('data/revdictionary.json', 'w')
file.write(json.dumps(revdictionary))
file.close()

# B) Topic Modeling (LDA)
- It identifies the hidden thematic structures within the documents and represent them as latent topics.
- Each document is a mixture of all possible topics with varying probabilities
- Each topic is a mixture of all vocabulary of the dataset with varying probabilities

*Setting random seeds*

In [6]:
# For reproducible results
random.seed(41)  # For Python random
np.random.seed(41)  # For NumPy random

## 1. Latent Dirichlet Allocation (LDA)
**LDA class**
main functions are:
1. Markov chain monte carlo initialization (giving the model a random inital state, expecting the model
    to converge for higher number of iterations.
2. Collapsed gibbs sampling inference: in each iteration \
   2.1 Iterates through all documents, all tokens/words in each document \
   2.2 For for each token computes its most suitable topic, given the current status of the model \
   2.3 Updates new topic if different from current topic, associated estimates update, so does the model state \
3. Estimate document-topic distribution from the final state of the model 
4. Estimate topic-word distribution (organized in decreasing order of probabilities) from the final state of the model
5. Other utility functions

In [7]:
# The class implements topic modeling (Latent dirichlet allocation) algorithm using collapsed gibbs sampling as in inference. 
class LDA:
    # topics to extract from the data (Components)
    _numTopics = None
    # vocabulary (unique words) in the dataset
    _arrVocab = None
    #size of vocabulary (count of unique words)
    _numVocabSize = None
    # dataset
    _arrDocs = []
    # dataset size (number of documents)
    _numDocSize = None
    # dirichlet prior (document to topic prior)
    _numAlpha = None
    # dirichlet prior (topic to word prior)
    _numBeta = None
    _ifScalarHyperParameters = True
    # Gibb sampler iterations
    _numGSIterations = None
    # The iterations for initial burnin (update of parameters)
    _numNBurnin = None
    # The iterations for continuous burnin (update of parameters)
    _numSampleLag = None
    
    
    
    # The following attributes are for internal working
    __numTAlpha = None  
    __numVBeta = None   
    __arrTheta = None
    __arrThetaSum = None
    __arrPhi = None
    __arrPhiSum = None
    __arrNDT = None
    __arrNDSum = []
    __arrNTW = None
    __arrNTSum = []
    __arrZ = []
    
    # for alpha to be a list, its size must be equal to the size of the dataset, has value for each doc
    # for beta to be a list, its size must be equal to the number of topics, has value for each topic  
    def __init__(self, numTopics = 2, numAlpha = 1.0, numBeta = 0.01, 
                 numGSIterations = 1000, numNBurnin = 50, numSampleLag = 20, 
                 wordsPerTopic = 10):
        self._numTopics = config["numTopics"]
        self._numAlpha = config["numAlpha"]
        self._numBeta = config["numBeta"]
        self._numGSIterations = config["numGSIterations"]
        self._numNBurnin = config["numNBurnin"]
        self._numSampleLag = config["numSampleLag"]
        self.__wordsPerTopic = config["wordsPerTopic"]
            
    #load data as integer encoding of words in a sequence (no padding or truncation)
    def getData(self, path):
        file = open(path, 'r')
        input_data = file.read()
        file.close()
        self.__loadData(input_data)
        self.__loadVocab()
        self.__prepareCollections()

    #load docs and docSize from the dataset
    def __loadData(self, input_data):
        rows = input_data.split('\n')
         
        #read dataset as documents of words IDs
        for row in rows:
            swordlist = row.split('\t')
            swordlist = list(filter(None, swordlist))   #remove empty items from list
            if len(swordlist) > 0:
                iwordlist = [eval(w) for w in swordlist]    
                self._arrDocs.append(iwordlist)

        # determine dataset size
        self._numDocSize = len(self._arrDocs)
        
        
    #Determine unique words (vocabulary) and count of unique words (vocabSize)    
    def __loadVocab(self):
        #determine unique vocabulary
        uniqueWords = []
        for doc in self._arrDocs:
            for word in doc:
                if word not in uniqueWords:
                    uniqueWords.append(word)
        self._arrVocab = uniqueWords
        self._numVocabSize = len(self._arrVocab)    

    def __prepareCollections(self):
        self.__arrNDSum = np.array([0] * self._numDocSize)
        self.__arrTheta = np.array([[0] * self._numTopics] * self._numDocSize)
        self.__arrThetasum = np.array([[0] * self._numTopics] * self._numDocSize)
        self.__arrNDT = np.array([[0] * self._numTopics] * self._numDocSize)
        
        self.__arrNTSum = np.array([0] * self._numTopics)
        self.__arrPhi = np.array([[0] * self._numVocabSize] * self._numTopics)
        self.__arrPhisum = np.array([[0] * self._numVocabSize] * self._numTopics)
        self.__arrNTW = np.array([[0] * self._numVocabSize] * self._numTopics)

        #Assign values to parameters based on hyper-parameters
        self.__numTAlpha = self._numTopics*self._numAlpha  
        self.__numVBeta = self._numVocabSize*self._numBeta   

        
        for d in range(0, self._numDocSize):
            rowOfZeros = [0] * len(self._arrDocs[d])
            self.__arrZ.append(rowOfZeros)
                
    # Initialize first markov chain randomly
    def randomMarkovChainInitialization(self):
        
        for d in range(self._numDocSize):
            wta = []                        #wta - word topic assignment
            doc = self._arrDocs[d]
            for ind in range(len(doc)): 
                randtopic = random.randint(0, self._numTopics - 1)      # generate a topic number at random
                self.__arrZ[d][ind] = randtopic
                self.__arrNDT[d][randtopic] += 1
                self.__arrNDSum[d] += 1
                wordid = self._arrDocs[d][ind]
                self.__arrNTW[randtopic][wordid] += 1
                self.__arrNTSum[randtopic] += 1
            
    
    #Inference (Collapsed Gibbs Sampling)
    def gibbsSampling(self):
        tAlpha = self._numAlpha * self._numTopics
        vBeta = self._numBeta * self._numVocabSize            
                    
        for it in range(self._numGSIterations):
            for d in range(self._numDocSize):
                dsize = len(self._arrDocs[d])
                for ind in range(dsize):
                    # remove old topic from a word instance
                    oldTopic = self.__arrZ[d][ind]
                    wordid = self._arrDocs[d][ind]
                    self.__arrNDT[d][oldTopic] -= 1
                    self.__arrNDSum[d] -= 1
                    self.__arrNTW[oldTopic][wordid] -= 1
                    self.__arrNTSum[oldTopic] -= 1   

                    # find a new more appropriate tpoic for the word instanc as per current state of the model
                    prob = [0] * self._numTopics
                    
                    for t in range(self._numTopics):
                        prob[t] = ((self.__arrNDT[d][t] + self._numAlpha) / (self.__arrNDSum[d] + tAlpha)) * \
                            (self.__arrNTW[t][wordid] + self._numBeta) / (self.__arrNTSum[t] + vBeta)
                    
                    #cumulate multinomial
                    cdf = prob
                    for x in range(1, len(cdf)):
                        cdf[x] += cdf[x-1]
                    
                    cutoff = random.random() * cdf[-1]
                    newTopic = 0
                    for i in range(len(cdf)):
                        if cdf[i] > cutoff:
                            newTopic = i
                            break
                    #update as per new topic
                    self.__arrZ[d][ind] = newTopic
                    self.__arrNDT[d][newTopic] += 1
                    self.__arrNDSum[d] += 1
                    self.__arrNTW[newTopic][wordid] += 1
                    self.__arrNTSum[newTopic] += 1
                
    def getTopicsPerDocument(self):
        results = ''
        results += "***Topics per Document***\n"
        doc_topic_dist = {f"topic-{t}": [] for t in range(self._numTopics)}
        for d in range(self._numDocSize):
            results += "Document " + str(d) + ":\n"
            for t in range(self._numTopics):
                val = (self.__arrNDT[d][t]+self._numAlpha)/(self.__arrNDSum[d]+self.__numTAlpha)
                results += "Topic " + str(t) + ":" + str(val) + '\t'
                doc_topic_dist[f"topic-{t}"].append(val)
            results += '\n'
        print(results)
        file = open('data/output-data/document-topic-distribution.txt', 'w')
        file.write(results)
        return doc_topic_dist
    
    def getWordsPerTopic(self, revdictionary):
        results = "***Words per Topic***\n"
        topic_word_dist = {}
        for t in range(self._numTopics):
            results += "\nTopic " + str(t) + ":"
            #flag = 0
            wpt = {}
            for v in range(self._numVocabSize):
                val = (self.__arrNTW[t][v]+self._numBeta)/(self.__arrNTSum[t]+self.__numVBeta)
                wpt[revdictionary[str(v)]] = float(val)
             #   flag += 1
             #   if flag == self.__wordsPerTopic:
             #       break
            results += '\n'
            wpt_sorted = sorted(wpt.items(), key=lambda x: x[1], reverse=True)[:self.__wordsPerTopic]
            topic_word_dist[f"topic-{t}"] = wpt_sorted
            for item in wpt_sorted:
                results += str(item)
        file = open('data/output-data/topic-word-distribution.txt', 'w')
        file.write(results)
        print(results)
        return topic_word_dist
    
    def printall(self):
        print("topics: ", self._numTopics)
        print("dataset: ", self._arrDocs)
        print("dataset size: ", self._numDocSize)
        print("vocab: ", self._arrVocab)
        print("vocab size: ", self._numVocabSize)
        print("ndt: ", self.__arrNDT)
        print("ndsum: ", self.__arrNDSum)
        print("ntw: ", self.__arrNTW)
        print("ntsum: ", self.__arrNTSum)
        print("z: ", self.__arrZ)

## 2. Run the model

In [8]:
if __name__ == "__main__":
    lda = LDA()
    lda.getData(config["integer-encoded-path"])
    lda.randomMarkovChainInitialization()
    lda.gibbsSampling()

## 3. Results

Topic modeling has two important results
1. **Latent topics** identified in the corpus. Each topic is represented by top most presentable words for that topic. Its similar to clustering in the sense that the words are grouped as topics and labeled unintuitively as topic 0, topic 1 etc. However, unlike clustering, the words have probabilities of relevance to the other words of the topic. Using these probabilities, only the top few words (10 or 20) are used to represent a topic. Therefore, it is also called *word topic distribution*. 

2. **Topics in documents** are the probabilities of topics within each document. A general conception is that a document is not about entirely about a single topic and instead has different percentages of multiple topics. The topics in documents provide the probabilities of each topic in each document.

To observe the output generated by this method closely, please goto `data/output-data/` folder having `word-topic-distribution.txt` and `document-topic-distribution.txt` 

**words distribution per topic** \
The three latent topics determind from this dataset are labeled as Topic 0, Topic 1, and Topic 2. \
*Topic 0:* Represents profit, production and growth in economy \
*Topic 1:* Represents trade and and deals of fuel and BMW mentioning Germany and France \
*Topic 2:* Represents jobs, Yukos firm, India and Japan

These three topics gives a general idea of the topics covered in the data.

*Topic word distribution*

**Topic distribution per document** \
Each document talks about the topics identified to different extent. For example, document 0 can be 45% topic 0, 45% topic 1 and 10% topic 2. 

Therefore, it is important to know which documents are dominated by which topics, so that if a reader is interested in knowing particularly about topic 1 they can only read the documents where topic 1 is the major topic.

In [9]:
doc_topic_dist = lda.getTopicsPerDocument()
doc_topic_dist["Text"] = input_data[:100]

# Printing topic distribution for the top 10 documents
df_dtd = pd.DataFrame(doc_topic_dist, index = ["Doc "+str(i+1) for i in range(len(doc_topic_dist['topic-1']))])

df_dtd.to_csv("data/output-data/document-topic-distribution.tsv", sep="\t", index=False)

df_dtd.head()

***Topics per Document***
Document 0:
Topic 0:0.125	Topic 1:0.5	Topic 2:0.375	
Document 1:
Topic 0:0.375	Topic 1:0.25	Topic 2:0.375	
Document 2:
Topic 0:0.125	Topic 1:0.75	Topic 2:0.125	
Document 3:
Topic 0:0.375	Topic 1:0.25	Topic 2:0.375	
Document 4:
Topic 0:0.7142857142857143	Topic 1:0.14285714285714285	Topic 2:0.14285714285714285	
Document 5:
Topic 0:0.42857142857142855	Topic 1:0.2857142857142857	Topic 2:0.2857142857142857	
Document 6:
Topic 0:0.125	Topic 1:0.375	Topic 2:0.5	
Document 7:
Topic 0:0.25	Topic 1:0.5	Topic 2:0.25	
Document 8:
Topic 0:0.375	Topic 1:0.375	Topic 2:0.25	
Document 9:
Topic 0:0.14285714285714285	Topic 1:0.14285714285714285	Topic 2:0.7142857142857143	
Document 10:
Topic 0:0.1111111111111111	Topic 1:0.6666666666666666	Topic 2:0.2222222222222222	
Document 11:
Topic 0:0.42857142857142855	Topic 1:0.2857142857142857	Topic 2:0.2857142857142857	
Document 12:
Topic 0:0.5	Topic 1:0.375	Topic 2:0.125	
Document 13:
Topic 0:0.4444444444444444	Topic 1:0.1111111111111111	To

,topic-0,topic-1,topic-2,Text
Doc 1,0.125000,0.500000,0.375000,India calls for fair trade rules
Doc 2,0.375000,0.250000,0.375000,Sluggish economy hits German jobs
Doc 3,0.125000,0.750000,0.125000,Indonesians face fuel price rise
Doc 4,0.375000,0.250000,0.375000,Court rejects $280bn tobacco case
Doc 5,0.714286,0.142857,0.142857,Dollar gains on Greenspan speech


**Word distribution per topic**

The three latent topics determined from this dataset are labeled as Topic 1, Topic 2, and Topic 3.

Topic 1: May be broadly interpreted as election and politics from its top words

Topic 2: May be broadly interpreted as business deals

Topic 3: May be broadly interpreted as movies and showbiz

These three topics give a general idea of the topics covered in the data.

In [10]:
with open(config["integer-word-dict-path"], 'r') as file:
    revdictionary = json.load(file)
topic_word_distribution = lda.getWordsPerTopic(revdictionary)

twd = {}
for topic in topic_word_distribution.keys():
    words, probabilities = zip(*topic_word_distribution[topic])
    twd[topic + "-word"] = words
    twd[topic + "-word-probability"] = probabilities

df_twd = pd.DataFrame(twd, index = ["word " +str(i+1) for i in range(len(topic_word_distribution['topic-1']))])
df_twd.to_csv("data/output-data/topic-word-distribution.tsv", sep="\t", index = False)
df_twd

***Words per Topic***

Topic 0:
('economy', 0.039125056962437336)('profit', 0.03261506412342946)('growth', 0.026105071284421584)('gets', 0.026105071284421584)('sales', 0.019595078445413708)('hits', 0.01308508560640583)('2004', 0.01308508560640583)('loan', 0.01308508560640583)('production', 0.01308508560640583)('high', 0.01308508560640583)
Topic 1:
('deal', 0.04214290722950704)('fuel', 0.03513077624290021)('German', 0.021106514269686554)('BMW', 0.021106514269686554)('trade', 0.014094383283079725)('rise', 0.014094383283079725)('French', 0.014094383283079725)('record', 0.014094383283079725)('prices', 0.014094383283079725)('boost', 0.014094383283079725)
Topic 2:
('jobs', 0.024509504309027562)('profits', 0.024509504309027562)('Yukos', 0.024509504309027562)('firm', 0.024509504309027562)('Japan', 0.024509504309027562)('India', 0.01839740847136483)('hits', 0.01839740847136483)('new', 0.01839740847136483)('Japanese', 0.01839740847136483)('$280bn', 0.012285312633702094)


,topic-0-word,topic-0-word-probability,topic-1-word,topic-1-word-probability,topic-2-word,topic-2-word-probability
word 1,economy,0.039125,deal,0.042143,jobs,0.024510
word 2,profit,0.032615,fuel,0.035131,profits,0.024510
word 3,growth,0.026105,German,0.021107,Yukos,0.024510
word 4,gets,0.026105,BMW,0.021107,firm,0.024510
word 5,sales,0.019595,trade,0.014094,Japan,0.024510
word 6,hits,0.013085,rise,0.014094,India,0.018397
word 7,2004,0.013085,French,0.014094,hits,0.018397
word 8,loan,0.013085,record,0.014094,new,0.018397
word 9,production,0.013085,prices,0.014094,Japanese,0.018397
word 10,high,0.013085,boost,0.014094,$280bn,0.012285


*Document topic distribution*


*print all details:*
- Integer encoded dataset
- Final state of the model

In [11]:
  # prints everything - for debugging
lda.printall()

topics:  3
dataset:  [[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19], [20, 21, 22, 23], [24, 25, 26, 6], [27, 28, 29, 30, 31], [32, 33, 34, 35, 36], [3, 37, 7, 38, 39], [0, 40, 41, 42], [43, 12, 44, 45, 46, 47], [48, 49, 50, 51], [52, 53, 54, 55, 56], [57, 58, 59, 60, 61, 62], [63, 64, 65, 66, 67], [68, 69, 70, 71], [72, 73, 74, 75, 76], [77, 78, 7, 79, 80], [57, 81, 82, 83], [84, 85, 86, 87], [88, 89, 90, 47], [91, 92, 93, 94], [95, 96, 97, 98, 99, 9], [100, 101, 102, 103], [104, 105, 106, 9], [107, 108, 109, 110, 111], [112, 113, 114, 115, 116], [117, 118, 119, 120, 121], [122, 96, 123, 56], [124, 125, 56, 126], [127, 85, 128, 129], [130, 131, 132, 133, 134], [135, 115, 136, 137, 138], [26, 131, 139, 140, 141, 142], [143, 6, 144, 145], [146, 147, 148, 149], [150, 151, 152, 153, 49], [154, 155, 139, 156, 53], [127, 157, 83, 158], [159, 160, 161, 162], [163, 12, 164, 165], [166, 167, 168, 169], [170, 171, 172, 173], [174, 175, 176, 177, 178], [179, 47, 180